# I. 문제 정의 및 데이터 선정

와인 평가 텍스트를 기반으로 감정 분석  
-> 해당 와인을 평가한 사람이 좋게 평가했는지를 알 수 있다

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).




*   country : 와인이 생산된 국가
*   description : 와인 평가 텍스트
*   designation : 와인 이름
*   points : 와인 평가 점수(1~100)
*   price : 와인의 가격
*   province : 와인이 생산된 지방
*   region_1, region_2 : 구체적인 지역
*   taster_name, taster_twitter_handle : 와인 평가를 남긴 사람
*   title : 와인 평가 제목
*   variety : 와인 종류(레드 와인, 화이트 와인 등등...)
*   winery : 와인을 만든 와이너리



In [53]:
import pandas as pd
import numpy as np
import tensorflow as tf

df = pd.read_csv('/content/drive/MyDrive/AIB/Section4/winemag-data-130k-v2.csv')
df = df.iloc[:, 1:] # 기존 df 첫번째 column인 Unnamed:0 column을 삭제

df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


# II. 데이터 전처리

**기존 df에서 필요한 컬럼만 사용 -> description(평가), points(평점)**  
평점 컬럼을 이용하여 평균 기준 긍정(1), 부정(0)을 나타내는 감정(sentiment)라는 새로운 column을 생성

In [54]:
# 필요한 컬럼만 수집 -> description(평가), points(평점)
df = df.loc[:, ['description', 'points']]
df.head()

,description,points
0,"Aromas include tropical fruit, broom, brimston...",87
1,"This is ripe and fruity, a wine that is smooth...",87
2,"Tart and snappy, the flavors of lime flesh and...",87
3,"Pineapple rind, lemon pith and orange blossom ...",87
4,"Much like the regular bottling from 2012, this...",87


In [55]:
# 와인의 평점 평균을 기준으로 긍정(1), 부정(0)으로 나눔
mean = df['points'].mean() # 약 88.4471

df['sentiment'] = df['points']

def sent(x) :
  if x >= mean :
    return 1
  else :
    return 0

df['sentiment'] = df['sentiment'].apply(sent)
df['sentiment'].value_counts(normalize = True)

0    0.528579
1    0.471421
Name: sentiment, dtype: float64

In [56]:
df.head()

,description,points,sentiment
0,"Aromas include tropical fruit, broom, brimston...",87,0
1,"This is ripe and fruity, a wine that is smooth...",87,0
2,"Tart and snappy, the flavors of lime flesh and...",87,0
3,"Pineapple rind, lemon pith and orange blossom ...",87,0
4,"Much like the regular bottling from 2012, this...",87,0


**평가 내용을 feature, 감정을 label로 사용**

In [57]:
feature = df['description']
label = df['sentiment']

In [58]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

**정규표현식(regex), 내장함수를 통한 소문자 통일(lower), 표제어 추출(lemmatization)**

In [59]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

# 표제어 추출을 하는 함수를 작성
def get_lemma(sentence):
  lemmas = []
  for word in sentence:
    words = lemmatizer.lemmatize(word, 'v') # 명사, 동사를 설정안해주면 동사를 원형으로 잘 추출해내지 못하므로 일단 동사에 집중
    lemmas.append(words)

  # lemmas = [lemmatizer.lemmatize(word, 'v') for word in sentence]

  return lemmas

In [60]:
import re
from nltk.corpus import stopwords

swlist = stopwords.words('english') # nltk가 정의한 영어 불용어 리스트

# 전처리를 해주는 함수를 작성
def preprocessing(sentence):
  sentence = re.sub('[^a-zA-Z]', ' ', sentence) # 알파벳이 아닌 것에 대해서는 공백으로 대체
  sentence = sentence.lower() # 대문자 알파벳을 소문자로 
  sentence = [word for word in sentence.split() if word not in swlist] # 불용어 리스트에 없는 단어들만 리스트 형식으로 나타냄
  sentence = get_lemma(sentence) # 위에서 만든 get_lemma 함수를 이용하여 표제어 추출

  return ' '.join(sentence) # 리스트 형태의 단어들을 문장으로 결합

In [62]:
# sample = df['description'][0]

# print(sample)
# print(preprocessing(sample))

Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.
aromas include tropical fruit broom brimstone dry herb palate overly expressive offer unripened apple citrus dry sage alongside brisk acidity


In [63]:
feature_preprocessed = feature.apply(preprocessing)

**학습셋과 테스트셋을 분리**

In [64]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(feature_preprocessed, label, test_size = 0.2, random_state = 42, stratify = label)

**Tokenizer를 이용하여 데이터 변환**

In [65]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

vocab_size = len(tokenizer.word_index) + 1 # 패딩이 0으로 들어가있기 때문에 +1 을 해주어야 한다
print(vocab_size)

23401


In [66]:
x_train_encoded = tokenizer.texts_to_sequences(x_train)
x_test_encoded = tokenizer.texts_to_sequences(x_test)

# 패딩 처리하기 전 maxlen 값을 결정하기 위해 학습 데이터에 있는 문서의 평균 토큰 수 구하기
print(f'학습 데이터에 있는 문서의 평균 토큰 수: {np.mean([len(sent) for sent in x_train_encoded], dtype=int)}')

학습 데이터에 있는 문서의 평균 토큰 수: 25


In [67]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 학습 데이터에 있는 문서의 평균 토큰 수가 25이므로 그것보다 조금 더 긴 50으로 설정
maxlen = 50

x_train = pad_sequences(x_train_encoded, maxlen = maxlen, truncating = 'post', padding = 'post')
x_test = pad_sequences(x_test_encoded, maxlen = maxlen, truncating = 'post', padding = 'post')

# 모델링

In [68]:
# 모델 학습 과정에서 같은 파라미터에서 일정한 성능을 내기 위해 시드값을 고정
np.random.seed(42)
tf.random.set_seed(42)

In [73]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

**word2vec**  
구글 뉴스 말뭉치로 학습된 word2vec 벡터 다운

In [80]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


vocab에 속하는 단어에 대해서 word2vec의 임베딩 가중치 행렬 생성

In [84]:
embedding_matrix = np.zeros((vocab_size, 300))

# 해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
def get_vector(word):
    if word in wv:
        return wv[word]
    else:
        return None
 
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp

embedding_matrix.shape

(23401, 300)

**모델 설계 & 컴파일**

In [86]:
model = Sequential()
model.add(Embedding(vocab_size, 300, input_length = maxlen, weights = [embedding_matrix], trainable = False))
# 이미 사전훈련된 벡터가 학습되지 않도록 trainable = False로 설정
model.add(LSTM(128, return_sequences = True))
model.add(LSTM(128))
model.add(Dropout(0.1))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['acc'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 50, 300)           7020300   
                                                                 
 lstm_8 (LSTM)               (None, 50, 128)           219648    
                                                                 
 lstm_9 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 32)                4128      
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                                 
Total params: 7,375,693
Trainable params: 355,393
Non-

**조기종료(Early Stopping 설정)**

In [87]:
# 학습시킨 데이터를 저장시키기 위한 코드(파라미터 저장 경로 설정)
checkpoint_filepath = "FMbest.hdf5"

# early stopping
early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 5, verbose = 1)
# 멈추는 기준(monitor)를 val_loss 로 설정해주고 loss가 Best 값보다 5번 이상 갱신이 안될 때 Stop(patience) 하도록 설정

# Validation Set을 기준으로 가장 최적의 모델을 찾기
save_best = ModelCheckpoint(filepath = checkpoint_filepath, monitor = 'val_loss', verbose = 1, 
                            save_best_only = True, save_weights_only = True, mode = 'auto', save_freq = 'epoch', options = None)
# Best 모델 역시 멈추는 기준을 val_loss로 하고 save_best_only=True, save_weights_only=True 로 설정

**모델 학습**

In [88]:
model.fit(x_train, y_train, batch_size = 100, epochs = 20,
          validation_data = (x_test, y_test), callbacks = [early_stop, save_best])

Epoch 1/20
1040/1040 [==============================] - ETA: 0s - loss: 0.4465 - acc: 0.7885
Epoch 1: val_loss improved from inf to 0.43881, saving model to FMbest.hdf5
1040/1040 [==============================] - 424s 403ms/step - loss: 0.4465 - acc: 0.7885 - val_loss: 0.4388 - val_acc: 0.7957
Epoch 2/20
1040/1040 [==============================] - ETA: 0s - loss: 0.4026 - acc: 0.8130
Epoch 2: val_loss improved from 0.43881 to 0.40425, saving model to FMbest.hdf5
1040/1040 [==============================] - 387s 372ms/step - loss: 0.4026 - acc: 0.8130 - val_loss: 0.4043 - val_acc: 0.8076
Epoch 3/20
1040/1040 [==============================] - ETA: 0s - loss: 0.3840 - acc: 0.8229
Epoch 3: val_loss improved from 0.40425 to 0.39546, saving model to FMbest.hdf5
1040/1040 [==============================] - 390s 375ms/step - loss: 0.3840 - acc: 0.8229 - val_loss: 0.3955 - val_acc: 0.8168
Epoch 4/20
1040/1040 [==============================] - ETA: 0s - loss: 0.3681 - acc: 0.8314
Epoch 4: va

**테스트셋에 대해 모델 평가**

In [89]:
# 학습 과정에서 가장 성능이 좋았던 모델을 불러옴
model.load_weights(checkpoint_filepath)

# 평가
model.evaluate(x_test, y_test)

813/813 [==============================] - 42s 52ms/step - loss: 0.3701 - acc: 0.8315


[0.3701074719429016, 0.831506073474884]

# 모델 저장

**인메모리 상태에서 학습된 모델을 다른 노트에서 사용할 수 있도록 따로 저장**

In [94]:
from keras.models import load_model

model.save('/content/drive/MyDrive/AIB/Section4/pj4model.h5')

모델 이외에도 전처리에 사용되는 tokenizer를 저장

In [103]:
import json

with open('/content/drive/MyDrive/AIB/Section4/wordIndex.json', 'w') as json_file:
    json.dump(tokenizer.word_index, json_file)